In [12]:
from typing import TypedDict, Annotated, Sequence
from typing_extensions import TypedDict

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, FunctionMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain import hub
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Your further code goes here


In [5]:
import re
import time
from pprint import pprint

from gsheet_handler import GSheetHandler
import importlib
module = importlib.import_module('gsheet_handler')
importlib.reload(module)
ggl = module.GSheetHandler('google_sheet_key.json')

data_raw = pd.read_csv('/home/sftpuser1/uploads/data.csv')
data_raw

no docx


,link,reference_time,article_time,title,text,author,tags
0,https://mmk.ru:443/ru/press-center/news/predst...,2024-05-30 11:43:19.735012,2024-05-29 00:00:00,Представители ММК выступили на Металлургическо...,Представители Магнитогорского металлургическог...,NaN,NaN
1,https://mmk.ru:443/ru/press-center/news/mmk-st...,2024-05-30 11:43:19.785068,2024-05-28 00:00:00,ММК стал лидером нового рейтинга промышленного...,Магнитогорский металлургический комбинат занял...,NaN,NaN
2,https://mmk.ru:443/ru/press-center/news/mmk-st...,2024-05-30 11:43:19.791400,2024-05-28 00:00:00,ММК стал победителем конкурса Российского союз...,Магнитогорский металлургический комбинат вошел...,NaN,NaN
3,https://mmk.ru:443/ru/press-center/news/trubna...,2024-05-30 11:43:19.797557,2024-05-23 00:00:00,Трубная отрасль показала наибольшую динамику з...,Из всех партнеров Магнитки представители трубн...,NaN,NaN
4,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.397327,2023-11-28 00:00:00,ArcelorMittal signs contract with Sarralle to ...,ArcelorMittal Asturias has signed a contract w...,NaN,Decarbonisation
5,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.398262,2023-11-16 00:00:00,ArcelorMittal announces the first industrial p...,ArcelorMittal announces the first industrial p...,NaN,Sustainability
6,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.504985,2023-11-15 00:00:00,Schneider Electric partners with ArcelorMittal...,"ArcelorMittal, the world’s leading steel and m...",NaN,"Sustainability,Sustainable Development"
7,https://corporate.arcelormittal.com/media/news...,2024-05-29 12:53:23.506656,2023-10-30 00:00:00,Update regarding Kostenko mining tragedy,Update regarding Kostenko mining tragedy\n ...,NaN,NaN
8,https://evrazsteel.ru/events/will/obrazovateln...,2024-05-29 23:18:00.759010,2024-06-06 00:00:00,"Образовательный интенсив «Удобство, скорость и...",Фото: Shutterstock/FOTODOM\n 6 июня 2024 го...,NaN,"проектирование,развивайся_с_нами"
9,https://evrazsteel.ru/events/will/vebinar-odno...,2024-05-29 23:18:00.798357,2024-05-30 00:00:00,Вебинар «Одностадийное проектирование с пример...,Фото: Shutterstock/FOTODOM\n 30 мая 2024 го...,NaN,"проектирование,развивайся_с_нами"


In [9]:
# main.py
import json
import os
import telebot

# Загрузка конфигурации из JSON-файла
with open('config.json') as config_file:
    config = json.load(config_file)

# Установка переменных окружения
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="nlmk_bot"

In [22]:
from datetime import datetime
from langchain import hub

def evaluate_news_article(link, nes_text, news_data, news_topic):
    llm = ChatOpenAI(temperature=0, model="gpt-4o")
    prompt = hub.pull("mlenparrot/nlmk_grade")
    today = datetime.now().strftime('%Y-%m-%d')
    params = {
        'link': link,
        'nes_text': nes_text,
        'news_data': news_data,
        'news_topic': news_topic,
        'today': today
    }
    chain = prompt | llm
    result = chain.invoke(params)
    return result

result = evaluate_news_article(
    link='https://example.com/news-article',
    nes_text='This is a sample news text.',
    news_data='2024-06-01',
    news_topic='Sample News Topic'
)

#print(result)

In [23]:
def apply_evaluation(row):
    result = evaluate_news_article(
        link=row['link'],
        nes_text=row['text'],
        news_data=row['article_time'],
        news_topic=row['title']
    )
    return pd.Series(result)

In [27]:
from tqdm import tqdm
tqdm.pandas()
data_news = data_raw.copy()
evaluation_results = data_news.progress_apply(apply_evaluation, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:21<00:00,  1.57s/it]


In [30]:
data_news_rated = pd.concat([data_news, evaluation_results], axis=1).drop(['reference_time', 'author', 'tags'], axis=1)
data_news_rated

,link,article_time,title,text,Source_Authority,Relevance_for_NLMK,Novelty_and_Relevance,Impact_on_the_Industr_and_Market
0,https://mmk.ru:443/ru/press-center/news/predst...,2024-05-29 00:00:00,Представители ММК выступили на Металлургическо...,Представители Магнитогорского металлургическог...,4,5,4,4
1,https://mmk.ru:443/ru/press-center/news/mmk-st...,2024-05-28 00:00:00,ММК стал лидером нового рейтинга промышленного...,Магнитогорский металлургический комбинат занял...,3,4,4,3
2,https://mmk.ru:443/ru/press-center/news/mmk-st...,2024-05-28 00:00:00,ММК стал победителем конкурса Российского союз...,Магнитогорский металлургический комбинат вошел...,4,5,4,3
3,https://mmk.ru:443/ru/press-center/news/trubna...,2024-05-23 00:00:00,Трубная отрасль показала наибольшую динамику з...,Из всех партнеров Магнитки представители трубн...,4,5,4,4
4,https://corporate.arcelormittal.com/media/news...,2023-11-28 00:00:00,ArcelorMittal signs contract with Sarralle to ...,ArcelorMittal Asturias has signed a contract w...,5,4,4,5
5,https://corporate.arcelormittal.com/media/news...,2023-11-16 00:00:00,ArcelorMittal announces the first industrial p...,ArcelorMittal announces the first industrial p...,5,4,5,4
6,https://corporate.arcelormittal.com/media/news...,2023-11-15 00:00:00,Schneider Electric partners with ArcelorMittal...,"ArcelorMittal, the world’s leading steel and m...",5,4,4,4
7,https://corporate.arcelormittal.com/media/news...,2023-10-30 00:00:00,Update regarding Kostenko mining tragedy,Update regarding Kostenko mining tragedy\n ...,5,4,4,5
8,https://evrazsteel.ru/events/will/obrazovateln...,2024-06-06 00:00:00,"Образовательный интенсив «Удобство, скорость и...",Фото: Shutterstock/FOTODOM\n 6 июня 2024 го...,3,4,5,3
9,https://evrazsteel.ru/events/will/vebinar-odno...,2024-05-30 00:00:00,Вебинар «Одностадийное проектирование с пример...,Фото: Shutterstock/FOTODOM\n 30 мая 2024 го...,3,4,5,3


In [14]:
prompt

StructuredPrompt(input_variables=['link', 'nes_text', 'news_data', 'news_topic', 'today'], metadata={'lc_hub_owner': 'mlenparrot', 'lc_hub_repo': 'nlmk_grade', 'lc_hub_commit_hash': '4412538d0d52c4b90a8c55489dd9d59d1d64196d6b6eafa7ade722c10ad770d9'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['today'], template='Какую проблему решаем?\nИТ подразделения НЛМК нуждаются в оперативной информации о новейших продуктах, инициативах конкурентов, инновациях в системах хранения и управления данными, ии, стартапах, законодательных требованиях, конференциях, открытиях и других событиях, которые могут повлиять на стратегию компании.\n\nЧто необходимо сделать?\nВыделение самой важной информации о новых продуктах, инициативах конкурентов, инновациях и других событиях.\n\nЦелевая аудитория\nИТ подразделения НЛМК, руководители и первые лица компаний, заинтересованные в оперативной информации о рынке и конкурентах.\n\nМетрики успеха проекта\nТочность сбора информации.\n

In [31]:
ggl.update_gsheet_with_df(data_news_rated,
                          'https://docs.google.com/spreadsheets/d/1cIB0yjbP-uInz7k0rs7zRruyct7C3Cks3P7DUGZTFlc/edit?usp=sharing', 
                          0)